<a href="https://colab.research.google.com/github/ruebot/notebooks/blob/main/yfile_indigenous_mallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cribbed from https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/09-Topic-Modeling-Without-Mallet.html

In [3]:
%%capture

import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version   
install_java()

!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [4]:
%%capture
!pip install tomotopy
!pip install little_mallet_wrapper
!pip install seaborn
!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

In [6]:
import tomotopy as tp
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
path_to_mallet = '/content/mallet-2.0.8/bin/mallet'

In [7]:
%%capture
!wget https://www.dropbox.com/s/mlabd2b3181esgp/corpus.zip
!unzip corpus.zip

--2022-11-03 20:40:45--  https://www.dropbox.com/s/mlabd2b3181esgp/corpus.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/mlabd2b3181esgp/corpus.zip [following]
--2022-11-03 20:40:45--  https://www.dropbox.com/s/raw/mlabd2b3181esgp/corpus.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e47482bc18e8a421b51a21bfd.dl.dropboxusercontent.com/cd/0/inline/BwF0NASSdzNYxBmdHUgBFV5zslP5yUln-e_odh3SWsHbzXDynfl_ziyj_s9IG7gIPCagg7EUXrTsGXTqKEElCLDGTR4x6cWRAcMkLsbhlIrcWcCbvgrSKKcS-pjo2AallsDsLjqmyae-ElTYrZcerCfiv40lF35ckaY-UBBWOo1IlA/file# [following]
--2022-11-03 20:40:46--  https://uc4e47482bc18e8a421b51a21bfd.dl.dropboxusercontent.com/cd/0/inline/BwF0NASSdzNYxBmdHUgBFV5zslP5yUln-e_odh3SWsHbzXDynfl_ziyj_s9IG7gIPCagg7EUXrTsGXTqKEElCL

In [9]:
files = glob.glob(f"corpus/*.txt")

In [10]:
training_data = []
original_texts = []
titles = []

for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove')
    training_data.append(processed_text)
    original_texts.append(text)
    titles.append(Path(file).stem)

In [11]:
len(training_data), len(original_texts), len(titles)

(84, 84, 84)

In [12]:
# Number of topics to return
num_topics = 15
# Numer of topic words to print out
num_topic_words = 10

# Intialize the model
model = tp.LDAModel(k=num_topics)

# Add each document to the model, after splitting it up into words
for text in training_data:
    model.add_doc(text.strip().split())
    
print("Topic Model Training...\n\n")
# Iterate over the data 10 times
iterations = 10
for i in range(0, 100, iterations):
    model.train(iterations)
    print(f'Iteration: {i}\tLog-likelihood: {model.ll_per_word}')

print("\nTopic Model Results:\n\n")
# Print out top 10 words for each topic
topics = []
topic_individual_words = []
for topic_number in range(0, num_topics):
    topic_words = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    topics.append(topic_words)
    topic_individual_words.append(topic_words.split())
    print(f"✨Topic {topic_number}✨\n\n{topic_words}\n")

Topic Model Training...


Iteration: 0	Log-likelihood: -9.092964525456892
Iteration: 10	Log-likelihood: -8.865508122937936
Iteration: 20	Log-likelihood: -8.756833223531238
Iteration: 30	Log-likelihood: -8.703925430978297
Iteration: 40	Log-likelihood: -8.65696447154206
Iteration: 50	Log-likelihood: -8.627939229173103
Iteration: 60	Log-likelihood: -8.617927982251345
Iteration: 70	Log-likelihood: -8.58753396923978
Iteration: 80	Log-likelihood: -8.561051253595839
Iteration: 90	Log-likelihood: -8.543500949140617

Topic Model Results:


✨Topic 0✨

indigenous community reconciliation nations first council school also people take

✨Topic 1✨

les nous sur autochtone aux dans communauté residential qui peuples

✨Topic 2✨

studies social york phd work canadian international new language political

✨Topic 3✨

des autochtones york pour vice dion cote meek president initiatives

✨Topic 4✨

york indigenous university students knowledge program new faculty student support

✨Topic 5✨

teaching learning

In [14]:
topic_distributions = [list(doc.get_topic_dist()) for doc in model.docs]

In [15]:
from IPython.display import Markdown, display
import re

def make_md(string):
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index, n=5):
    
    sorted_data = sorted([(_distribution[topic_index], _document) 
                          for _distribution, _document 
                          in zip(topic_distributions, docs)], reverse=True)
    
    topic_words = topics[topic_index]
    
    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n---")
    
    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words.split():
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        
        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc}\n\n')
    
    return

In [16]:
get_top_docs(titles, topic_distributions, topic_index=0, n=6)

### ✨Topic 0✨

indigenous community reconciliation nations first council school also people take

---

✨  
**Topic Probability**: 0.4165996313095093  
**Document**: 2021-05-31-a-statement-on-national-**indigenous**-month-and-the-discovery-at-the-kamloops-indian-residential-**school**



✨  
**Topic Probability**: 0.39976170659065247  
**Document**: 2021-09-29-a-statement-from-the-**indigenous**-**council**-on-this-day-for-truth-and-**reconciliation**



✨  
**Topic Probability**: 0.37773582339286804  
**Document**: 2022-06-21-cora-cody-appointed-**indigenous**-teaching-and-learning-librarian



✨  
**Topic Probability**: 0.36496755480766296  
**Document**: 2017-06-21-york-universitys-hart-house-renamed-to-create-safe-space-for-**indigenous**-peoples



✨  
**Topic Probability**: 0.3077191710472107  
**Document**: 2022-06-17-reflecting-on-national-**indigenous**-peoples-day-and-national-**indigenous**-peoples-month



✨  
**Topic Probability**: 0.2953163981437683  
**Document**: 2021-10-03-national-day-for-truth-and-**reconciliation**-panel-focuses-on-**reconciliation**-in-action



In [17]:
get_top_docs(titles, topic_distributions, topic_index=1, n=5)

### ✨Topic 1✨

les nous sur autochtone aux dans communauté residential qui peuples

---

✨  
**Topic Probability**: 0.5176928043365479  
**Document**: 2022-09-21-national-day-for-truth-and-reconciliation-events-planned-for-university-community



✨  
**Topic Probability**: 0.5124576091766357  
**Document**: 2022-09-28-a-message-on-the-national-day-for-truth-and-reconciliation



✨  
**Topic Probability**: 0.37045660614967346  
**Document**: 2021-06-20-a-statement-from-vice-president-equity-people-and-culture-sheila-cote-meek-on-national-indigenous-peoples-day



✨  
**Topic Probability**: 0.2986435890197754  
**Document**: 2022-05-31-national-indigenous-history-month-at-york



✨  
**Topic Probability**: 0.22220031917095184  
**Document**: 2022-05-24-a-statement-on-yorks-commitment-to-the-indigenous-framework-and-decolonizing-research



In [18]:
get_top_docs(original_texts, topic_distributions, topic_index=1, n=5)

### ✨Topic 1✨

les nous sur autochtone aux dans communauté residential qui peuples

---

✨  
**Topic Probability**: 0.5176928043365479  
**Document**: The York University community is encouraged to reflect on the legacy of Canada’s **residential** school system, its impact on Indigenous Peoples and what it means for the present and future. 
There are three events planned to mark National Day for Truth and Reconciliation.
York community members are invited to attend a lecture by Algoma University Professor Paulette Steeves on Thursday, Sept. 29. Steeves is the Canada Research Chair in Healing at Reconciliation at Algoma University. There will be a breakfast for Indigenous graduate students and faculty members on Friday, Sept. 30, followed by a virtual panel discussion that will be open to all York community members. 
Thursday, Sept. 29  The guest lecture by Steeves is titled “Reclaiming and Rewriting Indigenous Histories of the Western Hemisphere (the Americas).” The lecture will be held in the Congregation Room on the fourth floor of the Second Student Centre on the Keele Campus. The lecture will run from 4 to 5 p.m. EST.  
Friday, Sept. 30 There will be a networking breakfast with Indigenous graduate students and faculty and Steeves. This event is by special invitation only and will take place in the Second Student Centre, from 8:30 to 10 a.m. EST. For more information, contact Faculty of Education Professor Susan Dion, associate vice-president, Indigenous Initiatives, by email at avpii@yorku.ca.
There will be a panel discussion following the breakfast that will be focused on the theme “Reflecting on the Legacies of Residential Schools: What it means for our present and our futures.” The panel discussion will be a virtual event and will take place from 11 a.m. to noon. It will be moderated by Dion and will feature reflections by York University community members.
And finally, the official 2022 Every Child Matters Orange t-shirts are available at the University Bookstore with all proceeds going to the Orange Shirt Society. Zoom backgrounds will also be available on the National Day for Truth & Reconciliation website and can be downloaded and uses for the day.
For more information on these and other events, and for resources to support individual learning, visit York’s National Day for Truth and Reconciliation website.
Cette journée est l’occasion de rendre hommage **aux** enfants **qui** n’ont jamais pu retourner chez eux et **aux** survivants des pensionnats ainsi qu’à leurs familles et leurs communautés. Nous encourageons la **communauté** de York à réfléchir aux conséquences du système des pensionnats **sur** **les** **peuples** autochtones et à ce que cela signifie pour le présent et l’avenir.    
Trois événements sont prévus pour marquer la Journée nationale de la vérité et de la réconciliation. 
Les membres de la **communauté** de York sont invités à assister à une conférence de la professeure Paulette Steeves, titulaire de la Chaire de recherche du Canada en histoire **autochtone** – vérité et réconciliation à l’Université Algoma. La conférence de Mme Steeves aura lieu le jeudi 29 septembre. Un déjeuner sera organisé pour **les** étudiantes et étudiants de cycle supérieur et les membres du corps professoral autochtones le vendredi 30 septembre; il serasuivi d’un débat virtuel ouvert à tous les membres de la communauté de York. 
Jeudi 29 septembre  La conférence de Mme Steeves s’intitule « Reclaiming and Rewriting Indigenous Histories of the Western Hemisphere (the Americas) ». Elle aura lieu de 16 h à 17 h HNE **dans** la Congregation Room au quatrième étage du Second Student Centre **sur** le campus Keele.    
Vendredi 30 septembre Déjeuner de réseautage avec des étudiants de cycle supérieur, des professeurs autochtones et Mme Steeves. Cet événement est sur invitation seulement; il aura lieu de 8 h 30 à 10 h (HNE) au Second Student Centre.  Pour plus d’informations, contactez la professeure de la Faculté d’éducation Susan Dion, vice-présidente associée aux initiatives autochtones, par courriel à avpii@yorku.ca.
Après le déjeuner de réseautage, il y aura un débat d’experts sur le thème « Reflecting on the Legacies of Residential Schools: What it means for our present and our futures ». Le panel de discussion sera un événement virtuel **qui** aura lieu de 11 h à 12 h. Il sera animé par Mme Dion et comprendra des réflexions des membres de la communauté de l’Université York. 
Des tee-shirts officiels orange Every Child Matters 2022 sont en vente à la Librairie de l’Université; tous les bénéfices seront versés à la Orange Shirt Society. Le personnel pourra également télécharger et utiliser des fonds d’écran Zoom pour cette journée.
Pour obtenir plus d’informations sur tous les événements et des ressources pour appuyer l’apprentissage individuel, visitez le site Web de la Journée nationale de la vérité et de la réconciliation de York à https://www.yorku.ca/about/national-day-for-truth-and-reconciliation/.




✨  
**Topic Probability**: 0.5124576091766357  
**Document**: Dear colleagues, 
On the second annual National Day for Truth and Reconciliation we recognize the legacies of **residential** schools, honour survivors, their families, intergenerational survivors and the children who never came home. We acknowledge the severity of the legacy of **residential** schools and how it contributed to a loss of loved ones, and a disruption of family and way of life, cultural traditions and language retention. At York University we are taking time to engage with and reflect on the truths of residential schools and the impact of colonialism.  
This day is a reminder of our responsibility to learn the history, to reflect on our goals of decolonization and our commitments made in the Indigenous Framework. While we have made progress, we acknowledge that there is still much work to be done.
This year at York, we are paying specific attention to the voices of Indigenous students by listening to and learning from their perspectives. We have an opportunity to come together as a community to better understand the legacies of residential schools, how Indigenous families survived, and the lingering implications for Indigenous students who have been living with the repercussions to today. Indigenous students have much to teach us about what this history, this day, this learning means to them and what they hope for their future, their families, communities, and nations.
We encourage everyone to join us for a panel event:  “Reflecting on the Legacies of Residential Schools: What it means for our present and our futures,” on Friday, Sept. 30, from 11 a.m. to 12:30 p.m. The discussion will be moderated by Associate Vice-President Indigenous Initiatives Susan D. Dion. Hearing the voices of Indigenous students are central to understanding the actions that students would like to see at York as we continue towards the journey of reconciliation and decolonization. The panel will also include non-Indigenous students who have stepped up to share their stories of what Truth and Reconciliation means to them. We will be able to discuss what this day of learning means and to reflect on how to engage as individuals, communities, and institutions in the path to reconciliation. These discussions are important for Canada, and for other nations around the world, as we strive to create an inclusive and equitable future for everyone.
We know that this day, as well as its events and messages, may be emotional for many Indigenous members of our community, especially those who are survivors or with close personal or family ties to experiences in the residential schools, and we encourage all of our community members to please be kind to one another. Support is available for survivors and those affected through the Indian Residential Schools Survivors Society at 1-800-721-0066 or on the 24-hour crisis line at 1-866-925-4419. There are also a variety of supports available to the York community, which are listed on the Mental Health and Wellness website and through the Employee and Family Assistance Program. 
For those who may be unable to join we encourage you reflect, acknowledge and better understand the harms done and history of the past, we invite you to review the resources on York’s National Day for Truth and Reconciliation website to learn more.
Thank you. Merci. Miigwech, Anushiik.
Rhonda LentonPresident and Vice-Chancellor
Sheila Cote-Meek  Vice-President Equity, People & Culture
Susan D. Dion  Associate Vice-President, Indigenous Initiatives
Chers collègues, chères collègues, 
En cette deuxième Journée nationale de la vérité et de la réconciliation, **nous** reconnaissons l’héritage des pensionnats autochtones et **nous** rendons hommage **aux** survivants et **aux** survivantes, à leurs familles, aux survivants intergénérationnels et aux enfants **qui** ne sont jamais rentrés à la maison. Nous reconnaissons le lourd héritage du système des pensionnats et la façon dont il a contribué à la perte d’êtres chers et à la perturbation du mode de vie familiale, des traditions culturelles et du maintien de la langue. À l’Université York, nous prenons le temps de nous engager et de réfléchir aux vérités des pensionnats et à l’impact du colonialisme. 
Ce jour, il nous incombe d’apprendre l’histoire, de réfléchir à nos objectifs de décolonisation et aux engagements pris **dans** le Cadre stratégique **autochtone**. Nous avons fait des progrès, mais nous reconnaissons qu’il reste encore beaucoup à faire.
Cette année, à York, nous accordons une attention particulière aux voix étudiantes autochtones en **les** écoutant et en apprenant de leurs points de vue. Nous avons l’occasion de nous rassembler en tant que **communauté** afin de mieux comprendre **les** répercussions des pensionnats, la façon dont les familles autochtones ont survécu et leurs implications profondes **sur** la population étudiante **autochtone** jusqu’à aujourd’hui. Les membres de la **communauté** étudiante autochtone ont beaucoup à nous apprendre **sur** la signification de cette histoire, de cette journée et sur cet apprentissage pour eux ainsi que sur leurs espoirs pour l’avenir, leurs familles, leurs communautés et leurs nations.
Nous incitons tout le monde à se joindre à nous pour un panel de discussion sur le thème : Reflecting on the Legacies of Residential Schools: What it means for our present and our futures, le vendredi 30 septembre, de 11 h à 12 h 30. La discussion sera animée par Susan D. Dion, Vice-présidente associée aux initiatives autochtones. Il est essentiel d’entendre les voix de la population étudiante autochtone pour comprendre les actions qu’elle aimerait voir mises en œuvre à York, tandis que nous poursuivons le chemin vers la réconciliation et la décolonisation. Le panel inclura aussi des étudiants et étudiantes non autochtones **qui** ont accepté de parler de la signification du terme Vérité et Réconciliation pour eux. Nous pourrons discuter de la signification de cette journée d’apprentissage et réfléchir à la manière de nous engager, en tant qu’individus, communautés et institutions, sur la voie de la réconciliation. Ces discussions sont importantes pour le Canada, et pour d’autres nations **dans** le monde, car nous nous efforçons de créer un avenir inclusif et équitable pour tous.
Nous savons que cette journée ainsi que ses événements et ses messages peuvent être chargés d’émotions pour de nombreux membres autochtones de notre communauté, tout particulièrement les personnes survivantes ou celles qui ont des liens personnels ou familiaux étroits avec les expériences vécues dans les pensionnats. Nous encourageons tous les membres de notre communauté à faire preuve de bienveillance les uns envers les autres. Les survivants et les personnes concernées peuvent obtenir du soutien auprès de la Indian Residential Schools Survivors Society en composant le 1-800-721-0066 ou le 1-866-925-4419 pour la ligne d’écoute téléphonique 24 heures sur 24. Il existe aussi de nombreuses possibilités de soutien pour la communauté de York : vous trouverez notamment une liste sur le site Santé mentale et bien-être à York et dans le cadre du Programme d’aide aux employés et à leur famille. 
Si vous ne pouvez pas vous joindre à nous, nous vous encourageons à réfléchir, à reconnaître et à mieux comprendre les préjudices subis et l’histoire. Nous vous invitons à consulter les ressources sur le site Journée nationale de la vérité et de la réconciliation pour en savoir plus.
Thank you. Merci. Miigwech, Anushiik.
Rhonda Lenton  Présidente et vice-chancelière
Sheila Cote-Meek  Vice-présidente de l’équité, des personnes et de la culture
Susan D. Dion  Vice-présidente associée aux initiatives autochtones




✨  
**Topic Probability**: 0.37045660614967346  
**Document**: Today marks the 25th anniversary of what is now known as Indigenous Peoples Day. This is a day for all Canadians to recognize and celebrate the unique history, diverse heritage and cultures, and outstanding contributions of First Nations, Inuit and Métis peoples.
This year, however, the celebration of the survival of many distinct heritages, languages, cultural practices and spiritual beliefs will be taking place under the shadow of the hundreds of children who did not survive the Residential School system. In particular, the discovery of a mass grave of 215 children at the former site of the Kamloops Indian Residential School, the subsequent discoveries in Manitoba and Saskatchewan, and those yet to come, are a stark reminder of the trauma inflicted on many Indigenous peoples across Canada. The ongoing impact of the Residential Schools has been very challenging for many Indigenous peoples as we struggle to reconcile this difficult news.
First, I would ask that as part of the celebration, you take the time to learn more about the history and continuing experiences of colonization of Indigenous Peoples in Canada, including but not limited to the Residential Schools and the Sixties Scoop. Secondly, I request your support in calling on government(s) to take the necessary steps and actions required to change the existing systems and sources of continued oppression of Indigenous Peoples, especially children.
First Nations, Métis and Inuit children continue to be disproportionately represented in child welfare systems across the country, and Indigenous children living on-reserve receive less money for health care and education than other children across Canada. There is also  a need for greater awareness and understanding about the reality and the root causes of the violence against Indigenous women, girls and 2SLGBTQQIA people – who experience violence at a rate that is disproportionately higher than the general Canadian population.
On this, the 25th anniversary of National Indigenous Peoples Day, let us first celebrate, then learn more and take action:
Donate to Indigenous scholarships or an Indigenous-led organization.
Write to elected representatives.
Attend Indigenous community events with respect and openness.
Read books, watch films or television programs and view art by Indigenous Peoples.
Support is available for survivors and those affected through the Indian Residential Schools Survivors Society at 1-800-721-0066 or on the 24-hour crisis line at 1-866-925-4419.
There are also a variety of supports available to the York community listed on the Mental Health and Wellness site and through the Employee and Family Assistance Program.
Let us celebrate the strength and resilience of Indigenous Peoples at York and across Canada today and every day.
Miigwech, merci and thank you.
Sheila Cote-Meek
Vice-President Equity, People and Culture
Nous célébrons aujourd’hui le 25e anniversaire de la Journée nationale des **peuples** autochtones. Cet événement permet à tous **les** Canadiens et Canadiennes de reconnaître et de célébrer la richesse de l’histoire et la diversité de l’héritage et du patrimoine, ainsi que **les** contributions exceptionnelles des **peuples** des Premières Nations, des Inuits et des Métis.
Cette année toutefois, cette célébration de la survie des héritages, des langues, des pratiques culturelles et des croyances spirituelles est assombrie par la nouvelle des centaines d’enfants **qui** ont perdu la vie en raison du système des pensionnats. En particulier, la découverte d’une fosse commune de 215 enfants **sur** le site d’un ancien pensionnat **autochtone** à Kamloops, les découvertes **qui** ont suivi au Manitoba et en Saskatchewan — et celles encore à venir —, sont un rappel brutal des traumatismes infligés **aux** peuples autochtones partout au Canada. Les pensionnats ont eu des répercussions dramatiques pour de nombreux peuples autochtones et **nous** avons tous du mal à gérer ces terribles nouvelles.
Tout d’abord, je vous demande de prendre le temps, **dans** le cadre de cette célébration, d’en apprendre davantage **sur** l’histoire et l’expérience des peuples autochtones au Canada, notamment sur la colonisation et sur les pensionnats et la rafle des années 1960. Je sollicite aussi votre soutien pour demander **aux** gouvernements de prendre les mesures et les dispositions nécessaires pour changer les systèmes en place et les sources d’oppression contre les peuples autochtones, surtout chez les enfants.
De nos jours, les enfants des Premières Nations, des Métis et des Inuits continuent à être surreprésentés **dans** les systèmes de protection de l’enfance partout au pays : les enfants autochtones qui vivent sur une réserve reçoivent moins d’argent pour leurs soins de santé et leur éducation que les autres enfants dans l’ensemble du Canada. Il est également important de connaître et de comprendre la réalité et les origines de la violence contre les femmes, les filles et les personnes 2ELGBTQQIA autochtones qui la subissent à un taux beaucoup plus élevé que le reste de la population canadienne.
En ce 25e anniversaire de la Journée nationale des peuples autochtones, célébrons d’abord, puis renseignons-**nous** et passons à l’action :
Faites un don pour les Bourses pour étudiants autochtones ou à des organisations dirigées par des Autochtones.
Écrivez à vos représentants élus.
Assistez aux événements de la **communauté** **autochtone** avec respect et ouverture d’esprit.
Lisez des livres, écoutez des films ou des émissions de télévision et admirez les œuvres d’art des peuples autochtones.
Les survivants des pensionnats autochtones et autres personnes touchées peuvent obtenir du soutien auprès d’Indian Residential Schools Survivors Society en composant le 1 (800) 721-0066 ou en appelant la ligne d’écoute 24 heures sur 24 au 1 (866) 925-4419.
La **communauté** de York offre aussi plusieurs services de soutien que vous pouvez trouver sur le site Mental Health and Wellness de York et par l’entremise du Programme d’aide aux employés et à la famille.
Célébrons la force et la résilience des peuples autochtones à York et partout au Canada, aujourd’hui et tous les jours.
Miigwech, merci et thank you.
Sheila Cote-Meek
Vice-présidente de l’équité, des personnes et de la culture




✨  
**Topic Probability**: 0.2986435890197754  
**Document**: June 1 marks the start of Indigenous History Month, a time when we acknowledge Indigenous Peoples – their history, cultures, and many contributions and achievements. It is a specific opportunity to learn about and engage with Indigenous knowledges, languages, practices and ways of being, and their importance to both York and Canadian society more broadly.
York has acted with intention in cultivating relationships and creating space and place for the Indigenous community. Working with the Indigenous Council and in response to recommendations made in the Indigenous Framework and Decolonizing Research Administration report, last year, we welcomed the first associate vice-president, Indigenous initiatives to our leadership team. This year, we also established the Centre for Indigenous Knowledges and Languages (CIKL) and awarded the first Indigenous postdoctoral Fellow with the inaugural Provost’s Postdoctoral Fellowships for Black and Indigenous Scholars. Additionally, the York University Indigenous Research Seed Fund, announced in May 22, awarded 10 scholars a total of $204,298.
York looks forward to strengthening relationships with the local Indigenous communities that surround our campuses and we acknowledge that there is a long journey towards our goal of decolonization and reconciliation.
We encourage you to seek knowledge about the ways in which Indigenous peoples have contributed to society and history through the resources that are available to you. Please also take the time to learn about the many Indigenous initiatives that are taking place at York.
We remain grateful for all opportunities to foster dialogue, awareness and action.
Thank you. Merci. Miigwech, Anushiik.
Rhonda LentonPresident and Vice-Chancellor
Sheila Cote-MeekVice-President Equity, People & Culture
Susan D. DionAssociate Vice-President, Indigenous Initiatives
Le Mois de l’histoire **autochtone** commence aujourd’hui et **nous** en profitons pour rendre hommage **aux** **peuples** autochtones, à leur histoire, à leurs cultures et à leurs nombreuses contributions et réalisations. C’est une occasion d’apprendre et de s’engager **dans** **les** connaissances, **les** langues, les pratiques et les réalités autochtones et de reconnaître leur importance pour York et pour la société canadienne en général.
York a agi délibérément en développant des relations et en créant un espace et un lieu pour la **communauté** **autochtone**. En collaboration avec le Conseil autochtone et en réponse **aux** recommandations formulées **dans** le Cadre autochtone et le rapport Decolonizing Research Administration, **nous** avons accueilli l’année dernière dans notre équipe de direction la toute première vice-présidente associée aux initiatives autochtones. Cette année, nous avons créé le Centre des savoirs et langues autochtones et attribué la première bourse postdoctorale autochtone dans le cadre du Programme de bourses postdoctorales de la rectrice pour les chercheuses et chercheurs noirs et autochtones. De plus, le Fonds de stimulation de la recherche autochtone à l’Université York, annoncé le 22 mai, a accordé un montant total de 204 298 $ à 10 chercheurs.
York se réjouit de renforcer ses relations avec les communautés autochtones locales situées tout autour de ses campus. Nous reconnaissons qu’il reste un long chemin à parcourir pour atteindre notre objectif de décolonisation et de réconciliation.
Nous vous encourageons à découvrir, grâce aux ressources à votre disposition, les façons dont les **peuples** autochtones ont contribué à notre société et à l’histoire. Prenez également le temps de vous renseigner **sur** les nombreuses initiatives autochtones **qui** ont lieu à York.
Nous accueillons avec gratitude toutes les occasions de favoriser le dialogue, la sensibilisation et l’action.
Thank you. Merci. Miigwech, Anushiik.
Rhonda LentonPrésidente et vice-chancelière
Sheila Cote-MeekVice-présidente de l’équité, des personnes et de la culture
Susan D. DionVice-présidente associée aux initiatives autochtones




✨  
**Topic Probability**: 0.22220031917095184  
**Document**: La version française suit la version anglaise.
Dear York community members,
Our society has benefited immensely from Indigenous knowledge, languages and ways of knowing and being. Indigenous people contribute to positive change in our communities, our environment and to the enrichment and well-being of society.
Indigenous scholars and students have made and continue to achieve tremendous success in research, teaching and learning, despite ongoing barriers rooted in deep colonial policies and practices that govern research administration and in many ways conflict with Indigenous research methodologies. York acknowledges the implications of the lack of attention and recognition that was given to the legitimacy of Indigenous knowledge in the past.
York is committed to supporting emerging and established Indigenous scholars and the knowledge creation they are involved with, in collaboration with the community, knowledge keepers and elders. This commitment has been and continues to be an ongoing journey. York was the first Canadian university where a dissertation was written and defended in an Indigenous language. We are committed to putting in the work to support and recognize the legitimacy of Indigenous knowledge and the value that Indigenous scholars contribute. We recognize the significance of Indigenous languages and the knowledge embedded in language.
In November 2017, York launched the Indigenous Framework, which identified 10 principles the University needed to take action on to advance the goals of the Truth and Reconciliation Commission. Guided by the Framework, and working in consultation with the Indigenous Council, the Office of the Vice-President Research & Innovation developed a Decolonizing Research Administration Report to address the University’s research administration practices and policies, and to identify further steps the University should take to better support and foster Indigenous scholars.
The University has taken positive steps in this journey to date.
We would like to commend the Indigenous Council, the Office of the Vice-President Research & Innovation and the Division of Equity, People and Culture for their progress to date, and for their unwavering commitment to supporting Indigenous research and the work of Indigenous scholars.
The University is fully committed to taking action to decolonize research, and to recognizing and supporting research in service of the sovereignty and self-determination goals of Indigenous people, now and in the future. We will foster and promote scholarly activity relevant to Indigenous life, while respecting Indigenous approaches to knowledge and learning. By doing so, we will live our values by championing diversity and by creating a supportive, inclusive and caring environment for all.
The decolonization of research is not a short-term goal and there is still a great distance to go, but York is committed to putting in the work. We will continue to consult and collaborate with Indigenous scholars and students to build spaces and places for Indigenous knowledges and languages.
Sincerely,
Rhonda LentonPresident and Vice-Chancellor
Amir AsifVice-President, Research & Innovation
Sheila Cote-Meek, PhDVice-President, Equity, People & Culture
Chers membres de la **communauté** de York,
Notre société a énormément bénéficié des savoirs, des langues et des modes de connaissance et de vie autochtones. Les **peuples** autochtones contribuent à un changement positif **dans** nos communautés, notre environnement et à l’enrichissement et au bien-être de la société.
Les membres autochtones de la **communauté** de recherche et de la communauté étudiante ont obtenu et continuent d’obtenir d’immenses succès **dans** **les** domaines de la recherche, de l’enseignement et de l’apprentissage. Ils y parviennent malgré des obstacles permanents **qui** trouvent leur origine dans **les** politiques et pratiques coloniales profondément ancrées **qui** régissent l’administration de la recherche et qui, à bien des égards, entrent en conflit avec les méthodologies de recherche autochtones. York reconnaît les implications du manque de sensibilisation et de considération accordé à la légitimité des connaissances autochtones dans le passé.
York s’engage à soutenir les chercheuses et chercheurs autochtones émergents et établis ainsi que la création de connaissances à laquelle ils et elles participent, en collaboration avec la communauté, avec les Gardiens du savoir et avec les Aînés. Cet engagement était et demeure un cheminement permanent. York a été la première université canadienne où une thèse a été rédigée et soutenue dans une langue **autochtone**. Nous **nous** engageons à faire le nécessaire pour appuyer et pour reconnaître la légitimité des connaissances autochtones et la valeur de la contribution des chercheurs autochtones. Nous reconnaissons l’importance des langues autochtones et des savoirs qu’elles véhiculent.
En novembre 2017, York a lancé le Cadre stratégique **autochtone**, qui identifiait dix principes que l’Université devait mettre en œuvre pour faire avancer les objectifs de la Commission de vérité et de réconciliation. S’inspirant de ce cadre et en consultation avec le Conseil autochtone, le Bureau du vice-président de la recherche et de l’innovation a élaboré un Rapport **sur** la décolonisation de l’administration de la recherche afin d’examiner les pratiques et les politiques de l’Université en matière d’administration de la recherche et de déterminer les mesures supplémentaires que l’Université devrait adopter pour mieux soutenir et encourager les chercheurs autochtones.
À ce jour, l’Université a pris des mesures positives dans ce sens :
Nous tenons à féliciter le Conseil autochtone, le Bureau du vice-président de la recherche et de l’innovation et la Division de l’équité, des personnes et de la culture pour les progrès réalisés à ce jour et pour leur engagement indéfectible envers la recherche autochtone et le travail des chercheurs autochtones.
L’Université s’engage pleinement à prendre des mesures pour décoloniser la recherche ainsi qu’à reconnaître et à soutenir la recherche au service des objectifs de souveraineté et d’autodétermination des **peuples** autochtones, aujourd’hui et à l’avenir. Nous favoriserons et ferons la promotion d’activités savantes pertinentes pour la vie autochtone tout en respectant les approches autochtones de la connaissance et de l’apprentissage. Ce faisant, **nous** vivrons nos valeurs en défendant la diversité et en créant un environnement favorable, inclusif et bienveillant pour tout le monde.
La décolonisation de la recherche n’est pas un objectif à court terme. Il reste encore beaucoup de chemin à parcourir, mais York s’engage à y mettre du sien. Nous continuerons à consulter les universitaires autochtones et les membres de population étudiante autochtone et à collaborer avec eux afin de créer des espaces et des lieux pour les connaissances et les langues autochtones.
Sincères salutations,
Rhonda LentonPrésidente et vice-chancelière
Amir AsifVice-président de la recherche et de l’innovation
Sheila Cote-Meek, Ph. D.Vice-présidente de l’équité, des personnes et de la culture


